# Lane Detection on a Single Image

This notebook demonstrates lane detection on a single image using OpenCV and NumPy. The workflow includes cropping the region of interest, masking, thresholding, blurring, edge detection, and line detection to highlight lane lines in a road image.

---

## 1. Import Required Libraries
The following libraries are used for image processing and manipulation: NumPy, Matplotlib, OpenCV, and imutils.

In [4]:
import numpy as np
import matplotlib
import cv2
import imutils

## 2. Crop Region of Interest
The `snip_image` function crops the lower part of the image to focus on the road area where lane lines are expected.

In [2]:
def snip_image(image):
	red = (0,0,255)
	snip = image[(image.shape[0]-300):(image.shape[0]-50),(0):(image.shape[1])]
	return snip

## 3. Apply Polygonal Mask
The `mask_image` function applies a polygonal mask to the image, isolating the region where lane lines are likely to appear.

In [3]:
def mask_image(image):
	mask = np.zeros((image.shape[0],image.shape[1]), dtype = "uint8")

	pts = np.array([[90,image.shape[0]-25],[90,image.shape[0]-35],[int(image.shape[1]/2)-30,100], 
		[int(image.shape[1]/2)+30,100],[image.shape[1]-90, image.shape[0]-35],
		[image.shape[1]-90, image.shape[0]-25]],dtype=np.int32)
	
	for point in pts:
		cv2.circle(image, tuple(point), 4, (0,255,0), -1)
	cv2.fillConvexPoly(mask, pts,255)
	
	masked = cv2.bitwise_and(image,image,mask=mask)
	return masked

## 4. Thresholding for Lane Lines
The `thresh_image` function converts the image to HSV and grayscale, then applies a mask to highlight white lane markings.

In [5]:
def thresh_image(image):
	hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)

	gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
	
	whiteLower = np.array([30,0,65])
	whiteUpper = np.array([255,255,150])
	white_mask = cv2.inRange(hsv, whiteLower, whiteUpper)
	frame = cv2.bitwise_or(gray, white_mask)
	threshold = 100 
	black_white = cv2.threshold(frame, threshold, 255, cv2.THRESH_BINARY)[1]
	return white_mask

## 5. Gaussian Blurring
The `blur_image` function applies Gaussian blur to the image to reduce noise and smooth the result.

In [6]:
def blur_image(image):


	return cv2.GaussianBlur(image, (15,15),0)

## 6. Edge Detection
The `edge_image` function uses the Canny algorithm to detect edges in the blurred image, which helps in identifying lane boundaries.

In [7]:
def edge_image(image):

	return cv2.Canny(image,30,130)

## 7. Line Detection
The `lined_image` function applies the Hough Transform to detect straight lines in the edge-detected image, corresponding to lane markers.

In [8]:
def lined_image(image):

	return cv2.HoughLines(image, 1, np.pi/180,20)
	

## 8. Lane Detection Pipeline
The `main` function orchestrates the lane detection process: reading the image, applying all processing steps, and displaying the result with detected lane lines.

In [ ]:
def main():
	image = cv2.imread("images/20190219.jpg")
	#print(image.shape)
	# cv2.imshow("lane detect", image)

	snip = snip_image(image)
	# print(snip.shape)
	mask = mask_image(snip)
	
	blur = blur_image(mask)
	frame = thresh_image(mask)
	blur = blur_image(frame)
	edged= edge_image(blur)
	
	lines = lined_image(edged)
	
	# cv2.imshow("threshold", frame)
	
	# cv2.imshow("snip", snip)
	
	# cv2.imshow("mask", mask)
	
	# cv2.imshow("blurred", blur)
	# cv2.imshow("edged",edged)


	#print(lines)
	
	

	if lines is not None:
		pt1L = (0,0)
		pt2L = (0,0)
		pt2R = (0,0)
		pt1R = (0,0)
		L = 0
		R = 0
		for i in range(0,len(lines)):
			rho = lines[i][0][0]
			theta = lines[i][0][1]
			
			a = np.cos(theta)
			b = np.sin(theta)
			
			x0 = a*rho
			y0 = b*rho
			
			pt1 = ( int(x0 + 1000*(-b)), int(y0+1000*(a)))
			pt2 = ( int(x0 - 1000*(-b)), int(y0-1000*(a)))

			if pt2[1] < 0:
				pt1L = tuple(map(sum,zip(pt1L,pt1)))
				pt2L = tuple(map(sum,zip(pt2L,pt2)))
				L +=1
			else:
				pt1R = tuple(map(sum,zip(pt1R,pt1)))
				pt2R = tuple(map(sum,zip(pt2R,pt2)))
				R +=1

	
	
	cv2.line(snip, tuple(map(lambda x: int(x/L), pt1L)),tuple(map(lambda x: int(x/L), pt2L)), (0,0,255), 1)
	cv2.line(snip, tuple(map(lambda x: int(x/R), pt1R)),tuple(map(lambda x: int(x/R), pt2R)), (255,0,0), 1)
	
	cv2.imshow("lined", snip)
	cv2.waitKey(0)

## 9. Run the Lane Detection Pipeline
Call the `main()` function to execute the lane detection workflow and visualize the detected lane lines on the image.

In [10]:
main()

(480, 640, 3)
[[[ -91.            2.1118484 ]]

 [[ -93.            2.1293018 ]]

 [[ -85.            2.1118484 ]]

 [[ -99.            2.1293018 ]]

 [[ -72.            2.0769417 ]]

 [[ -81.            2.0943952 ]]

 [[-104.            2.146755  ]]

 [[-110.            2.1642082 ]]

 [[-102.            2.146755  ]]

 [[ -75.            2.0943952 ]]

 [[ -78.            2.0943952 ]]

 [[-113.            2.1642082 ]]

 [[-108.            2.146755  ]]

 [[ -63.            2.0594885 ]]

 [[-120.            2.1816616 ]]

 [[ -67.            2.0594885 ]]

 [[ -65.            2.0594885 ]]

 [[-122.            2.1816616 ]]

 [[ -56.            2.042035  ]]

 [[ -54.            2.042035  ]]

 [[ -52.            2.042035  ]]

 [[-128.            2.1991148 ]]

 [[ -59.            2.042035  ]]

 [[-132.            2.1991148 ]]

 [[-126.            2.1991148 ]]

 [[ -44.            2.024582  ]]

 [[ -41.            2.024582  ]]

 [[ -49.            2.024582  ]]

 [[-141.            2.2165682 ]]

